In [2]:
import os
import pandas as pd
import numpy as np
import rasterio
import geopandas as gpd
from rasterstats import zonal_stats
from multiprocessing import Pool, cpu_count
from functools import partial
import glob
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# If not installed, run this cell. It is needed for watershed delineation
#!pip install rasterstats
#!pip show rasterstats

In [3]:
# Configuration
root_folder = r"Z:\PhD_Datasets&Analysis\Info_Inputs"
css_folder = root_folder + "\\Streamflow_Sts_Drainage_Areas\GRDC_Watersheds"
tam_out_dir = r"Z:\PhD_Datasets&Analysis\Outputs\T&M_WBM"
tc_ds = root_folder + "\\TerraClimate"
out_geotiff = tc_ds + "\\GeoTIFF"
serial_id = 'grdcno_int'
tc_vars = ["ppt", "pet", "q"] # variable names according to TerraClimate
wb_var = 'wyield2' # Change this to the variable you want to process, e.g., 'wyield' for water yield
sql_field = "has_monthl" # Field to filter rows with monthly k recessions
#sql_field = "has_daily_" # Field to filter rows with daily k recessions

In [ ]:
# Processing parameters
#years = range(1958, 2023 + 1)
years = range(1970, 2023 + 1)
months = range(1, 12 + 1)
n_processes = min(cpu_count() - 1, 50)  # Use available cores minus 1, max 8

In [5]:
# Setup directories
processing_dir = os.path.join(tam_out_dir, wb_var)
if wb_var in ["bflow2", "wyield2"]:
    wb_var_clean = wb_var[:-1]  # Remove the last character '2'
else:
    wb_var_clean = wb_var

# Note: raster_dir not needed anymore since we're using the vector layer directly

In [7]:
def load_station_polygons():
    """
    Load station drainage area polygons from the existing vector layer.
    Returns a dictionary with station IDs as keys and geometries as values.
    """
    print("Loading station drainage areas from vector layer...")
    station_polygons = {}
    
    # Drainage areas vector layer
    drain_areas_file = os.path.join(css_folder, "CSS-WATERSHEDS-MERGE_FINAL_SELECTION.shp")  # Adjust file name as needed

    if not drain_areas_file:
        raise FileNotFoundError(f"Could not find drainage areas vector file in {css_folder}")
    
    try:
        print(f"Loading from: {drain_areas_file}")
        
        # Load the drainage areas vector layer
        gdf = gpd.read_file(drain_areas_file)
        
        print(f"Loaded vector layer with {len(gdf)} features")
        print(f"Available columns: {list(gdf.columns)}")
        
        # Check if the serial_id field exists
        if serial_id not in gdf.columns:
            raise ValueError(f"Field '{serial_id}' not found in drainage areas layer. Available fields: {list(gdf.columns)}")
        
        # Create dictionary with station IDs as keys and geometries as values
        for idx, row in gdf.iterrows():
            if row[sql_field] != 'Yes':
                continue
            station_id = str(row[serial_id])  # Convert to string to ensure consistency
            station_polygons[station_id] = row['geometry']
        
        print(f"Loaded {len(station_polygons)} station drainage areas")
        
        # Print first few station IDs for verification
        if station_polygons:
            sample_ids = list(station_polygons.keys())[:5]
            print(f"Sample station IDs: {sample_ids}")
        
        return station_polygons
        
    except Exception as e:
        print(f"Error loading drainage areas vector layer: {e}")
        raise

In [8]:
def get_raster_path(year, month, wb_var_clean, tc_vars, processing_dir, out_geotiff):
    """Get the appropriate raster file path based on variable type."""
    if wb_var_clean not in tc_vars:
        # Use processed variable from T&M model
        raster_path = os.path.join(processing_dir, f"{wb_var_clean}_{year}_{month}.tif")
    else:
        # Use original GeoTIFF from TerraClimate
        raster_path = os.path.join(out_geotiff, f"{wb_var_clean}_{year}_{month}.tif")
    
    return raster_path

In [9]:
def calculate_zonal_stats_single(args):
    """
    Calculate zonal statistics for a single station, year, and month combination.
    Each polygon is processed independently to handle overlapping features.
    """
    station_id, year, month, station_data, wb_var_clean, tc_vars, processing_dir, out_geotiff = args
    
    try:
        # Extract geometry from station data
        geometry = station_data['geometry']
        
        # Get raster path
        raster_path = get_raster_path(year, month, wb_var_clean, tc_vars, processing_dir, out_geotiff)
        
        # Check if raster file exists
        if not os.path.exists(raster_path):
            print(f"Warning: Raster file not found: {raster_path}")
            return None
        
        # Calculate zonal statistics for this individual polygon
        # Use a list with single geometry to ensure independent processing
        stats = zonal_stats(
            [geometry],  # Process as individual geometry
            raster_path, 
            stats=['mean', 'count'],
            geojson_out=False,
            nodata=np.nan,
            all_touched=False  # Only pixels with center inside polygon
        )
        
        if stats and len(stats) > 0:
            stat = stats[0]
            if stat['mean'] is not None and stat['count'] is not None:
                return {
                    serial_id: int(station_id),
                    'YEAR': year,
                    'MONTH': month,
                    'COUNT': stat['count'],
                    'MEAN': stat['mean']
                }
    
    except Exception as e:
        print(f"Error processing station {station_id}, year {year}, month {month}: {e}")
        
    return None

In [10]:
def save_results(df, year, wb_var_clean, tc_vars, processing_dir, out_geotiff):
    """Save results to CSV file."""
    print(f"\tSaving zonal statistics results for year {year}...")
    
    if wb_var_clean not in tc_vars:
        output_path = os.path.join(processing_dir, f"{wb_var_clean}_zonal_statistics_{year}.csv")
    else:
        output_path = os.path.join(out_geotiff, f"{wb_var_clean}_zonal_statistics_{year}.csv")
    
    # Ensure output directory exists
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    
    df.to_csv(output_path, index=False)
    print(f"\tSaved to: {output_path}")

In [11]:
def process_year_batch(year_batch, station_polygons, wb_var_clean, tc_vars, processing_dir, out_geotiff, n_processes):
    """
    Process a batch of years with parallelization.
    Each polygon is processed independently to handle overlapping features.
    """
    #all_results = []
    
    for year in year_batch:
        print(f"\n**Processing year {year}**")
        
        # Prepare arguments for parallel processing
        args_list = []
        station_ids = list(station_polygons.keys())
        
        # Process each station independently
        for station_id in station_ids:
            station_data = station_polygons[station_id]
            for month in months:
                args_list.append((
                    station_id, year, month, station_data, 
                    wb_var_clean, tc_vars, processing_dir, out_geotiff
                ))
        
        print(f"\tCalculating zonal statistics for {len(station_ids)} stations and {len(months)} months...")
        print(f"\tTotal operations: {len(args_list)}")
        print(f"\tUsing {n_processes} processes...")
        print(f"\tNote: Each polygon processed independently to handle overlaps")
        
        # Process in parallel
        with Pool(processes=n_processes) as pool:
            results = pool.map(calculate_zonal_stats_single, args_list)
        
        # Filter out None results and convert to DataFrame
        valid_results = [r for r in results if r is not None]
        
        if valid_results:
            df_year = pd.DataFrame(valid_results)
            #all_results.append(df_year)
            
            print(f"\tProcessed {len(valid_results)} valid records for year {year}")
            
            # Save results for this year
            save_results(df_year, year, wb_var_clean, tc_vars, processing_dir, out_geotiff)
        else:
            print(f"\tNo valid results for year {year}")
    
    #return pd.concat(all_results, ignore_index=True) if all_results else pd.DataFrame()
    return


In [12]:
def main():
    """
    Main processing function.
    Process a batch of years with parallelization.
    Each polygon is processed independently to handle overlapping features.

    """
    print('############################################################')
    print('\t\tINITIAL VARIABLES')
    print(f'\tPeriod to be executed: {years[0]}-{years[-1]}')
    print(f'\tVariable: {wb_var_clean}')
    print(f'\tProcesses: {n_processes}')
    print('\tNote: Each polygon processed independently to handle overlaps')
    print('############################################################')
    
    # Load station drainage areas
    station_polygons = load_station_polygons()
    
    if not station_polygons:
        print("No station polygons loaded. Exiting.")
        return
    
    # Process years in batches to manage memory
    batch_size = 5  # Process 5 years at a time
    year_batches = [list(years)[i:i+batch_size] for i in range(0, len(years), batch_size)]
    
    print(f"\nProcessing {len(years)} years in {len(year_batches)} batches...")
    print(f"Each of the {len(station_polygons)} polygons will be processed independently...")
    
    for i, year_batch in enumerate(year_batches, 1):
        print(f"\n{'='*60}")
        print(f"Processing batch {i}/{len(year_batches)}: {year_batch}")
        print(f"{'='*60}")
        
        try:
            process_year_batch(
                year_batch, station_polygons, wb_var_clean, 
                tc_vars, processing_dir, out_geotiff, n_processes
            )
            print(f"Completed batch {i}/{len(year_batches)}")
            
        except Exception as e:
            print(f"Error processing batch {i}: {e}")
            continue
    
    print("\nDONE!!")

In [ ]:
if __name__ == "__main__":
    
    # Process all polygons
    main()

############################################################
		INITIAL VARIABLES
	Period to be executed: 1970-2023
	Variable: wyield
	Processes: 7
	Note: Each polygon processed independently to handle overlaps
############################################################
Loading station drainage areas from vector layer...
Loading from: Z:\PhD_Datasets&Analysis\Info_Inputs\Streamflow_Sts_Drainage_Areas\GRDC_Watersheds\CSS-WATERSHEDS-MERGE_FINAL_SELECTION.shp
Loaded vector layer with 809 features
Available columns: ['grdc_no', 'river', 'station', 'area', 'altitude', 'lat_org', 'long_org', 'lat_pp', 'long_pp', 'dist_km', 'area_calc', 'quality', 'type', 'comment', 'source', 'grdcno_int', 'GRDCCOUNTR', 'Continent', 'has_monthl', 'has_daily_', 'monthly_k_', 'daily_k_re', 'Next_Downs', 'CATCHMENT_', 'Priority', 'geometry']
Loaded 808 station drainage areas
Sample station IDs: ['3617110', '3617811', '3617812', '3617814', '3618051']

Processing 54 years in 11 batches...
Each of the 808 polygons 